In [1]:
import pandas as pd
import numpy as np
import pickle
import time
from datetime import date

import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold,train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline

In [3]:
with open('train_cleaned.pickle','rb') as read_file:
    train = pickle.load(read_file)

In [4]:
train.head()

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,...,destination_id,destination_type_id,is_booking,similar_events,hotel_continent,hotel_country,hotel_market,hotel_cluster,stay_duration,plan_time
6,2,3,66,189,10067,2014.665587,501,0,0,2,...,8267,1,0,2,2,50,675,41,1.0,15.0
7,2,3,66,189,10067,2014.665587,501,0,1,2,...,8267,1,0,1,2,50,675,41,1.0,15.0
9,2,3,66,189,10067,2014.665587,501,0,0,2,...,8267,1,0,1,2,50,675,70,1.0,15.0
10,2,3,66,189,10067,2014.665587,501,0,0,2,...,8267,1,0,1,2,50,675,98,1.0,15.0
43,30,4,195,991,47725,2014.665587,1048,1,0,9,...,8803,1,0,1,3,151,1197,5,2.0,215.0


In [5]:
# xx = pd.Series([1,1,2,2,5])
# np_utils.to_categorical(xx, num_classes=5)

### Train, test, split

In [6]:
train['hotel_cluster'].value_counts()

91    55754
41    41797
48    39949
64    39626
5     32914
65    32783
98    32019
59    30603
70    30271
42    29740
Name: hotel_cluster, dtype: int64

In [7]:
train['hotel_cluster'].replace([91, 41, 48, 64, 5, 65, 98, 59, 70, 42], \
                               ['ninety-one','forty-one','forty-eight','sixty-four', 'five',\
                               'sixty-five','ninety-eight','fifty-nine','seventy','forty-two'], inplace=True)
train['hotel_cluster'].value_counts()

ninety-one      55754
forty-one       41797
forty-eight     39949
sixty-four      39626
five            32914
sixty-five      32783
ninety-eight    32019
fifty-nine      30603
seventy         30271
forty-two       29740
Name: hotel_cluster, dtype: int64

In [8]:
X = train.drop(['hotel_cluster'], axis=1)
y = train['hotel_cluster']

In [9]:
from sklearn.preprocessing import LabelEncoder

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
y = np_utils.to_categorical(encoded_y)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [11]:
# scales data between 0 and 1
X_train = keras.utils.normalize(X_train, axis=1)  
X_test = keras.utils.normalize(X_test, axis=1)

In [12]:
# y_test = np_utils.to_categorical(y_test, num_classes=10)
# y_train = np_utils.to_categorical(y_train, num_classes=10)

In [13]:
X_train.shape

(292364, 22)

### Keras

https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

In [ ]:
def c_model(optimizer):

  model = Sequential()
  model.add(Dense(2048, activation='relu', input_dim=22))
  model.add(Dense(512, activation='relu'))
  model.add(Dense(256, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(10, activation='softmax'))
  
  model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  
  return model

model = KerasClassifier(build_fn=c_model, epochs=30, batch_size=128)
parameters = {'optimizer':['RMSprop','Adagrad','Adam','Adamax']}
clf = GridSearchCV(model, parameters)
clf.fit(X_train, y_train)

/Users/kelseyheng/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
W0918 14:57:21.156636 4559062464 deprecation_wrapper.py:119] From /Users/kelseyheng/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0918 14:57:21.339053 4559062464 deprecation_wrapper.py:119] From /Users/kelseyheng/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0918 14:57:21.390038 4559062464 deprecation_wrapper.py:119] From /Users/kelseyheng/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.

Epoch 1/30
109824/194909 [===============>..............] - ETA: 33s - loss: 2.2566 - acc: 0.1673

In [ ]:
print(clf.best_score_, clf.best_params_)
means = clf.cv.results_['mean_tset_score']
parameters = clf.cv_results['params']
for mean, parameter in zip(means, parameters):
  print(mean, parameter)